# 线性回归

## 模型及求解思路

$y=\boldsymbol\omega \boldsymbol x+b$，其中$\boldsymbol \omega$和$\boldsymbol x$都是向量。

线性回归的神经网络图：
![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.1_linreg.svg)

### 损失函数

常见的损失函数：MSE（Mean Square Error),它评估第i个样本的损失为：
$$
l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2,
$$

**加了一个1/2是为了方便求导**

在训练集上的所有样本的平均损失为：
$$
{MSE}=L(\mathbf{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\mathbf{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} + b - y^{(i)}\right)^2.
$$

训练的目的就是使损失函数MSE降到最小

### 优化方法：随机梯度下降

- 小批量随机梯度下降（Mini-Batch SGD)

算法思想：

1）初始化参数 

2）进行多次迭代，在每次迭代中，随机选取一组样本(Batch size个)，组成小批量（Mini-Batch） ，求小批量中数据样本的平均损失关于模型参数的梯度（倒数），最后用此梯度与预先设定的一个正数（**学习率**）的乘积，作为模型参数在此次迭代中的减小量。

3）重复迭代，直至收敛（梯度变为0） 

$$
(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b)
$$

## 从零实现线性回归

In [1]:
# todo

# Softmax与分类模型

## softmax回归模型

softmax模型和线性回归模型都是将输入特征和权重做线性叠加，不同在于softmax的输出值个数等于标签里的类别个数。

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.4_softmaxreg.svg)
<center>softmax model 神经网络图<center>

但是，直接使用输出层的输出有两个问题：1.输出层的取值范围不确定；2.真实标签是离散值，这些离散值与不确定取值范围的输出的误差难以衡量。

所以我们采用了softmax operator : 


$$
 \hat{y}_1, \hat{y}_2, \hat{y}_3 = \text{softmax}(o_1, o_2, o_3) 
$$

其中

$$
 \hat{y}1 = \frac{ \exp(o_1)}{\sum_{i=1}^3 \exp(o_i)},\quad \hat{y}2 = \frac{ \exp(o_2)}{\sum_{i=1}^3 \exp(o_i)},\quad \hat{y}3 = \frac{ \exp(o_3)}{\sum_{i=1}^3 \exp(o_i)}. 
$$

容易看出$\hat{y}_1 + \hat{y}_2 + \hat{y}_3 = 1$且$0 \leq \hat{y}_1, \hat{y}_2, \hat{y}_3 \leq 1$，因此$\hat{y}_1, \hat{y}_2, \hat{y}_3$是一个合法的概率分布。这时候，如果$\hat{y}_2=0.8$，不管$\hat{y}_1$和$\hat{y}_3$的值是多少，我们都知道图像类别为猫的概率是80%。此外，我们注意到

$$
 \underset{i}{\arg\max} o_i = \underset{i}{\arg\max} \hat{y}_i 
$$

因此softmax运算不改变预测类别输出。



## 交叉熵损失函数

交叉熵：衡量两个概率分布差异的测量函数（真概率分布和预测概率分布）

第$i$个样本的交叉熵损失函数：
$$
H\left(\boldsymbol y^{(i)}, \boldsymbol {\hat y}^{(i)}\right ) = -\sum_{j=1}^q y_j^{(i)} \log \hat y_j^{(i)},
$$
$n$个样本的平均损失函数:
$$
\ell(\boldsymbol{\Theta}) = \frac{1}{n} \sum_{i=1}^n H\left(\boldsymbol y^{(i)}, \boldsymbol {\hat y}^{(i)}\right ),
$$

## 实现softmax

**为什么要梯度清零？**此部分参考了知乎用户[@Pascal](https://www.zhihu.com/question/303070254/answer/573037166)的回答

pytorch中，如果不进行梯度清零，计算的梯度会累加。

- 进行梯度清零的训练流程：

```python
for i,(images,target) in enumerate(train_loader):
    # 1. input output
    images = images.cuda(non_blocking=True)
    target = torch.from_numpy(np.array(target)).float().cuda(non_blocking=True)
    outputs = model(images)
    loss = criterion(outputs,target)

    # 2. backward
    optimizer.zero_grad()   # reset gradient
    loss.backward()
    optimizer.step()
```
例如上面的流程：每个batch的数据进行训练后都会进行梯度清零并反向传播，对参数进行优化。也就是说，每个batch都会更新网络。

- 不进行梯度清零的训练流程：

```python
for i,(images,target) in enumerate(train_loader):
    # 1. input output
    images = images.cuda(non_blocking=True)
    target = torch.from_numpy(np.array(target)).float().cuda(non_blocking=True)
    outputs = model(images)
    loss = criterion(outputs,target)

    # 2.1 loss regularization
    loss = loss/accumulation_steps   
    # 2.2 back propagation
    loss.backward()
    # 3. update parameters of net
    if((i+1)%accumulation_steps)==0:
        # optimizer the net
        optimizer.step()        # update parameters of net
        optimizer.zero_grad()   # reset gradient
 ```
 
 该流程中，

1.获取loss：输入图像和标签，通过infer计算得到预测值，计算损失函数；
 
2.loss.backward() 反向传播，计算当前梯度；

3.多次循环步骤1-2，不清空梯度，使梯度累加在已有梯度上；
 
4.梯度累加了一定次数后，先optimizer.step() 根据累计的梯度更新网络参数，然后optimizer.zero_grad() 清空过往梯度，为下一波梯度累加做准备；


# 多层感知机

## 多层感知机模型结构

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_mlp.svg)

在前面单层神经网络的基础上引入了一到多个隐藏层神经元，而且都是全连接层（dense layer)。

### 激活函数

为什么要引入激活函数，因为全连接层只是对输入做了仿射变换，多个仿射变换的叠加仍然是仿射变换，所以我们需要引入非线性的激活函数。

- ReLu函数

只保留正数元素，将负数变为0.

$\operatorname{ReLU}(x)=\max (x, 0)$


![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_relu.png)
<center>Relu图像</center>

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_relu_grad.png)

<center>Relu导数图像</center>


- sigmoid函数

sigmoid $(x)=\frac{1}{1+\exp (-x)}$

sigmoid函数可以将元素的值变换为0到1之间（循环神经网络中起到控制信息在神经网络中的流动的作用）。

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_sigmoid.png)
<center>sigmoid图像</center>

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_sigmoid_grad.png)
<center>sigmoid导数图像</center>


- tanh函数（双曲正切函数）

$\tanh (x)=\frac{1-\exp (-2 x)}{1+\exp (-2 x)}$ 

可以将元素的值变换为-1到1之间。(输入接近0时，tanh函数接近线性变换），形状和sigmoid函数很像，但是tanh函数在坐标系原点上对称。

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_tanh.png)
<center>tanh图像</center>

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.8_tanh_grad.png)
<center>tanh导数图像</center>

### 激活函数的选择

Relu是通用的，但是只能用于隐藏层。

用于分类器时，sigmoid函数及其组合效果更好，但是由于梯度消失问题，有时候要避免使用sigmoid函数和tanh函数。

神经网络层数较多时，最好使用Relu函数，简单而且计算量小。